# 6D scan

In [ ]:
import sys
import os
from os.path import join
from datetime import datetime
import importlib
from pprint import pprint
import json
import numpy as np
import pandas as pd
import h5py
import skimage
from tqdm import trange
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib import patches
import proplot as pplt

sys.path.append('..')
from tools import image_processing as ip
from tools import plotting as mplt
from tools import utils

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['cmap.discrete'] = False

## Load data 

In [ ]:
datadir = '/Diagnostics/Data/Measurements/scan6d/2021-11-30/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '211130223317-scan6d'

In [ ]:
# _filename = filename + '-scalars-only' + '.h5'
_filename = filename + '.h5'
file = h5py.File(join(datadir, _filename), 'r')
print(list(file))

In [ ]:
if 'config' in file:
    config = file['config']
    print(f"'config', {type(config)}")
    for key in config:
        print(f"  '{key}', {type(config[key])}")
        for name in config[key].dtype.names:
            print(f'    {name}: {config[key][name]}')
    # Make dictionary of metadata
    metadata = dict()
    for name in config['metadata'].dtype.names:
        metadata[name] = config['metadata'][name]
else:
    # Older measurement; metadata is in json file.
    metadata = json.load(open(join(datadir, filename + '-metadata.json'), 'r'))
    _metadata = dict()
    for _dict in metadata.values():
        for key, value in _dict.items():
            _metadata[key] = value
    metadata = _metadata
    print('metadata:')
    pprint(metadata)

In [ ]:
if 'log' in file:
    log = file['log']
    print(f"'log', {type(log)}")
    for item in log.dtype.fields.items():
        print('  ', item)

    print('\nErrors and warnings:')
    for i in range(log.size):
        if not(log[i, 'level'] == 'INFO'.encode('utf')):
            timestr = datetime.fromtimestamp(log[i, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
            print(f"{timestr} {log[i, 'message']}")
else:
    f = open(join(datadir, '211130223317-scan6d.txt'), 'r')
    for line in f:
        print(line.rstrip())
    f.close()

In [ ]:
data = file['scandata']

print(f"'scandata', {type(data)}")
for item in data.dtype.fields.items():
    print('  ', item)
print(f"nbytes = {data.nbytes:.3e}")

## Scan overview 

In [ ]:
acts = ['y_PositionSync', 'xp_PositionSync', 'x_PositionSync', 'yp_Readback', 'I_CurrentRb', 'phase_Set', 'steerI_Set']

In [ ]:
for act in acts:
    fig, ax = pplt.subplots(figsize=(10.0, 1.5))
    ax.plot(data[act], color='black')
    ax.format(xlabel='Point', ylabel=act, ygrid=False)
    plt.show()

## How big does an ellipsoid have to be to enclose the points?

Here are the slit ranges/correlations.

In [ ]:
slit_dict = {
    'y': {
        'center': 14.5,
        'distance': 20.0,
        'steps': 32,
    },
    'xp': {
        'center': 13.0,
        'distance': 11.0,
        'steps': 8,
    },
    'x': {
        'center': 13.0,
        'distance': 15.0,
        'steps': 8,
    },
    'yp': {
        'center': 67.5,
        'distance': 14.0,
        'steps': 8,
    },
    'I': {
        'center': 359.0,
        'distance': 14.0,
        'steps': 10,
    },
    'phase': {
        'center': 0.0,  # should be `caget(phaseset_name)`
        'distance': 0.0,
        'steps': 0,
    },
    'steerI': {
        'center': 0.0,  # should be `caget(steerset_name)`
        'distance': 0.0,
        'steps': 0,
    },
}
keys = ['y', 'xp', 'x', 'yp', 'I', 'phase', 'steerI']
center = np.array([slit_dict[key]['center'] for key in keys])
distance = np.array([slit_dict[key]['distance'] for key in keys])
steps = np.array([slit_dict[key]['steps'] for key in keys])

In [ ]:
ndim = len(slit_dict)
M = np.identity(ndim)  # (y1, x2, x1, y2 (bsm wire)
# 0 y1 (Slit_HZ04)
# 1 x2 (Slit_VT06)
# 2 x1 (Slit_VT04)
# 3 y2 (bsm wire)
# 4 I_dipole
# 5 phi_bsm
# 6 steerI_bsm

M[0, 3] = -0.35  # dy / dyp
M[1, 2] = 0.65  # dxp / dx
M[1, 4] = -0.15  # dxp / dI
M[2, 4] = -0.15  # dx / dI

M[5, 0] = 0.0  # dphi_bsm / dy
M[5, 1] = -3.0  # dphi_bsm / dxp
M[5, 2] = 12.0  # dphi_bsm / dx
M[5, 3] = 2.7  # dphi_bsm / dyp
M[5, 4] = 57.1  # dphi_bsm / dI

M[6, 1] = 0.0 # dsteerI / dxp 
M[6, 2] = -4.0  # dsteerI / dx
M[6, 4] = 0.0  # dsteerI / dI 

In [ ]:
profy = data['cam08_ProfileY'].copy()

In [ ]:
signal = profy.sum(axis=1)
signal = signal - np.min(signal)
signal[signal > np.mean(signal) + 1.0 * np.std(signal)] = 0
signal = signal / np.max(signal)
thresh = 0.05
valid, = np.where(signal >= thresh)

div = 10  # split scan into `div` parts
length = len(data) // div  # number of points per divigions
for yscale in [None, 'log']:
    for i in range(div):
        lo = i * length
        hi = (i + 1) * length
        idx = np.arange(lo, hi)
        
        _signal = signal[idx]
        _valid, = np.where(_signal >= thresh)
        
        fig, ax = pplt.subplots(figsize=(10.0, 2.0))
        ax.plot(
            _signal, color='lightgray',
            # marker='.', ms=2, ec='None', alpha=1, lw=0,
        )
        ax.plot(_valid, _signal[_valid], color='black', lw=0, marker='.', ms=2, ec='None')
        ax.format(xlabel='Point', ylabel='Signal')
        # ax.format(yscale=yscale, ylim=(-0.05, 1.0))
        plt.show()

In [ ]:
# for name in ['cam08_Integral', 'cam08_Saturation']:
#     fig, ax = pplt.subplots(figsize=(10.0, 1.5))
#     ax.plot(
#         data[name], color='lightgray',
#         # marker='.', ms=2, ec='None', alpha=1, lw=0,
#     )
#     ax.format(xlabel='Point', ylabel=name)
#     if 'Integral' in name:
#         ax.format(yscale='log')
#     plt.show()

In [ ]:
# div = 10  # split scan into `div` parts
# length = len(data) // div  # number of points per divigions
# pv = 'cam08_Integral'  

# signal = data[pv].copy()
# saturation = data['cam08_Saturation'].copy()

# rms = np.std(signal)
# thresh = 0.14
# valid = signal > thresh

# for i in range(div):
#     lo = i * length
#     hi = (i + 1) * length
#     idx = np.arange(lo, hi)
    
#     fig, ax = pplt.subplots(figsize=(10.0, 1.5))
#     _signal = signal[idx]
#     _valid = valid[idx]
#     ax.plot(idx, _signal, color='lightgray')
#     ax.plot(idx[_valid], _signal[_valid], color='black', marker='.', lw=0, ms=1)
#     ax.format(xlabel='Point', ylabel=pv)
#     ax.format(yscale='log')
#     # plt.savefig('_output/signal.png')
#     plt.show()

In [ ]:
n = 5  # dimensions to look at
acts = ['y_PositionSync', 'xp_PositionSync', 'x_PositionSync', 'yp_Readback', 'I_CurrentRb']
points = np.vstack([data[act] for act in acts[:n]]).T 
points_n = utils.apply(np.linalg.inv(M[:n, :n]), points - center[:n])  # remove correlations
points_nn = points_n / (0.5 * distance[:n])  # scale within unit box

In [ ]:
fig, ax = pplt.subplots()
dim1, dim2 = ('y1', 'y2')
i, j = [dims.index(dim) for dim in (dim1, dim2)]
ax.scatter(points_nn[:, i], points_nn[:, j],
           c='lightgray', s=3)
ax.scatter(points_nn[valid, i], points_nn[valid, j],
           c='black', s=3)
plt.show()

In [ ]:
fig, ax = pplt.subplots()
rs = []
rmaxs = []
ths = np.linspace(0.0, 10.0 * thresh, 300)
for th in ths:
    _valid = signal >= th
    rs.append(np.std(radii[_valid]))
    rmaxs.append(np.max(radii[_valid]))

ax.plot(ths, rs, label='std', color='black')
ax.format(ylabel='rstd')
ax2 = ax.twinx(color='blue6')
ax2.plot(ths, rmaxs, color='blue6')
ax2.format(ylabel='rmax')
ax.axvspan(0, thresh, color='lightgray', label='thresh')
ax.format(xlabel='thresh')
# plt.savefig('_output/rs.png')

In [ ]:
for yscale in [None, 'log']:
    fig, axes = pplt.subplots(ncols=5, figwidth=9.0)
    for th, ax in zip(np.linspace(0.0, 5.0, 5) * thresh, axes):     
        _valid = signal >= th
        ax.hist(radii, bins=bins, label='all', color='lightgrey')
        ax.hist(radii[_valid], bins=bins, label='above thresh', color='black')
        ax.format(title=f'{(th / thresh):.2f} thresh')
    axes.format(ylabel='num. points', xlabel='radius', yscale=yscale)
    axes[-1].legend(ncols=1, loc='r', framealpha=0)
    plt.savefig(f'_output/radii_sub_yscale{yscale}.png')
    plt.show()

In [ ]:
radii = np.sqrt(np.sum(np.square(points_nn), axis=1))

bins = 'auto'
for yscale in [None, 'log']:
    with pplt.rc.context(legendfontsize='medium'):
        fig, ax = pplt.subplots(figsize=(3, 1.85))
        ax.hist(radii, bins=bins, label='all', color='lightgrey')
        ax.hist(radii[valid], bins=bins, label='above thresh', color='black')
        ax.format(ylabel='num. points', xlabel='radius', yscale=yscale)
        ax.legend(ncols=2, loc='top', framealpha=0)
        plt.savefig(f'_output/radii_yscale{yscale}.png')
        plt.show()
        
rmax = np.max(radii[valid])
rmax

In [ ]:
rmax

In [ ]:
frac = np.count_nonzero(radii <= rmax) / len(radii)
print(frac)

frac_signal = float(len(signal[valid])) / len(signal)
print(f'{frac_signal:.3f}')

volume_ratio = utils.volume_sphere(n=5, r=rmax) / utils.volume_box(n=5, r=1.0)
print(volume_ratio)

In [ ]:
with pplt.rc.context(legendfontsize=9):
    fig, ax = pplt.subplots(figsize=(2.5, 2))
    ax.format(cycle='538')
    rs = np.linspace(1.0, 2.0)


    for n in [2, 3, 4, 5, 6]:
        ax.plot(rs, utils.volume_sphere(n=n, r=rs) / utils.volume_box(n=n, r=1.0), 
                # color='black'
                label=f'n={n}',
               )
    ax.legend(loc='lower right', ncols=1)
    ax.format(xlabel='n-sphere radius', ylabel='volume relative to box')
    ax.format(ylim=(0, 1))
    plt.savefig('_output/savings.png')
    plt.show()

In [ ]:
dims = ['y1', 'x2', 'x1', 'y2', 'I']
for _points, title in zip((points, points_n, points_nn), ('true', 'upright', 'upright + scaled')):
    fig, axes = pplt.subplots(ncols=n, nrows=n, figwidth=n*1.5, spanx=False, spany=False)
    axes.format(suptitle=title)
    for i in range(n):
        for j in range(n):
            axes[i, j].scatter(_points[:, j], _points[:, i], c='black', ec='None', s=1)
        axes[i, 0].format(ylabel=dims[i])
        axes[-1, i].format(xlabel=dims[i])
    plt.show()

In [ ]:
_points = points_nn
_rmax = rmax

fig, axes = pplt.subplots(ncols=5, nrows=5, figwidth=5.0, spanx=False, spany=False)
for i in range(5):
    for j in range(5):
        ax = axes[i, j]
        ax.scatter(_points[:, j], _points[:, i], c='lightgray', ec='None', s=0.5)
        ax.scatter(_points[valid, j], _points[valid, i], c='black', ec='None', s=0.5)
        if i != j:
            ax.add_patch(patches.Ellipse((0.0, 0.0), 2.0 * _rmax, 2.0 * _rmax, color='red', fill=False))
    axes[i, 0].format(ylabel=dims[i])
    axes[-1, i].format(xlabel=dims[i])
plt.savefig('_output/bounding_ellipse.png')
plt.show()